In [ ]:
"""
Author: Reggie Hyde
Date: Spring 2025
Description: This Python jupyter notebook replicates the results of
Investor-Tuned ML Portfolios with Turnover, Risk, and Arbitrage Difficulty Controls
The notebook is split into 2 main parts:
1) Replicating Gu et. al's NN3 model from Empirical Asset Pricing via Machine Learning (2020)
2) Proposing a framework for multi-objective portfolio optimization that accounts for practicality across investor types, driven by risk premia
predictions from Gu et. al's NN3 model.
"""

#Thank you! Please reach out with any questions at reginaldjhyde@gmail.com or hyde9698@stthomas.edu

#Library imports
import os #For saving checkpoints, accessing characteristic data, etc.
import glob #For file searching
import numpy as np #For matrix math
import pandas as pd #For tabular data
import gc #For manual garbage collecting for memory management (optional, might be useful for those with limited computational power)
import pickle #For saving data
from itertools import product #For using cartesian product

#Pytorch library imports
import torch #For training neural network
import torch.nn as nn #For accessing built-in common machine learning functions such as ReLU
import torch.optim as optim #For accessing built-in common machine learning optimizers (we use Adam)
from torch.utils.data import TensorDataset, DataLoader #For training data management
from torch.cuda.amp import autocast, GradScaler #For accessing mixed-precision training

#Scikit-learn imports
from sklearn.model_selection import train_test_split, ParameterGrid #For model selection functionality

print("Libraries imported successfully.")

In [ ]:
"""
THE FOLLOWING CELLS REPLICATE GU ET. AL'S NN3 NEURAL NETWORK
First, we must replicate their training, validation, and out-of-sample testing time window specifications.
Gu. et al implement a validation rolling window of sorts.
"""

validation_length = 12 #Define the length of the validation period, in years

years = np.arange(1987, 2017) #Create an array of years from 1987 to 2016 (end is non-inclusive)

estimation_periods = pd.DataFrame({ #Build a DataFrame with time period specifications
    'oos_year': years, #Out-of-sample year for testing
    'validation_end': years-1, #Validation period ends one year before the OOS year
    'validation_start': years - validation_length, #Validation starts 'validation_length' years before OOS year
    'training_start': 1957, #Training period starts from 1957
    'training_end': years - validation_length - 1 #Training ends one year before validation starts
})

print(estimation_periods.head()) #Print out the first 5 periods to verify replication

In [ ]:
class NN3(nn.Module): #This class defines the neural network model architecture for the NN3 model
    def __init__(self, input_dim):
        super(NN3, self).__init__()
        self.net = nn.Sequential( #Stacks layers in following order
            nn.Linear(input_dim, 32), #The network compresses the 920 interaction term inputs into 32 neurons in the first hidden layer
            nn.ReLU(), #ReLU activation function
            nn.Linear(32, 16), #The network compresses the 32 neurons in the first hidden layer into the 16 neurons in the second hidden layer
            nn.ReLU(), #ReLU activation function
            nn.Linear(16, 8), #The network compresses the 16 neurons in the second hidden layer into the 8 neurons in the third hidden layer
            nn.ReLU(), #ReLU activation function
            nn.Linear(8, 1) #The network compresses the 8 neurons in the third hidden layer into the output (predicted risk premia)
        )

    def forward(self, x): #Specifies forward pass
        return self.net(x)

print("NN3 model defined.")

In [ ]:
def train_model(model, train_loader, val_loader, lr, weight_decay, epochs=100): #This function trains the model for a given OOS year for a given combination of hyperparameters
    device = torch.device("cuda") #Find GPU. Note: Training this model on a CPU alone for all the OOS years will be a significant undertaking.
    model = model.to(device) #Designate GPU to train model

    criterion = nn.MSELoss() #Specify loss function as Mean Squared Error
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) #Specify optimizer as Adam with current hyperparameters
    scaler = torch.amp.GradScaler() #Specify scaler as gradient scaler for mixed-precision training

    #INITIAL TRAINING
    for epoch in range(epochs): #Iterate through n number of epochs, default will be 100
        model.train() #Specify that we're training the model
        running_loss = 0.0

        for xb, yb in train_loader: #Iterate through each small batch of input data and their corresponding outputs
            xb, yb = xb.to(device), yb.to(device) #Move batch of data onto the GPU
            optimizer.zero_grad() #Clear gradients from last backward pass

            with torch.amp.autocast("cuda"): #Specify automatic mixed precision to GPU
                loss = criterion(model(xb).squeeze(), yb) #Compute training loss

            scaler.scale(loss).backward() #Compute gradients for this batch
            scaler.step(optimizer) #Update weights based on these gradients
            scaler.update() #Dynamically adjust scaling factor

            running_loss += loss.item() #Add this batch's loss to total for later average calculation

        avg_loss = running_loss / len(train_loader) #Calculate average loss across all batches
        print(f"Epoch {epoch+1}/{epochs} - Training Loss: {avg_loss:.6f}")

    #VALIDATION (for evaluating best set of hyperparameters later)
    model.eval() #Specify that we're now validating the model
    val_losses = [] #Initialize empty array of batch validation losses
    with torch.no_grad(): #Turn off gradient tracking
        for xb, yb in val_loader: #Iterate through each small batch of validation data and their corresponding outputs
            xb, yb = xb.to(device), yb.to(device) #Move batch of data onto the GPU

            with torch.amp.autocast("cuda"): #Specify automatic mixed precision to GPU
                val_loss = criterion(model(xb).squeeze(), yb) #Compute validation loss

            val_losses.append(val_loss.item()) #Add this batch's loss to array for later average calculation

    return model, np.mean(val_losses) #Return the trained model and the validation loss average

print("Training function defined.")

In [ ]:
param_grid = { #Define the possible values for learning rate and weight decay, based on best practices (citations required)
    'learning_rate': [0.001, 0.01],
    'penalty': [1e-5, 1e-3],
}
grids = list(ParameterGrid(param_grid)) #Define the hyperparameter grid for learning rate and weight decay
print("Hyperparameter grid defined.")

In [ ]:
#ACKNOWLEDGEMENT: Characteristics data was prepared with the help of a Tidy Finance blog post by Stefan Voigt (https://www.tidy-finance.org/blog/gu-kelly-xiu-replication/)
#This cell loads all the prepared characteristic data
downloads_folder = r"PATH_OF_YOUR_PREPARED_CHARACTERISTICS" #Specify path of downloads folder
file_paths = glob.glob(os.path.join(downloads_folder, "characteristics_prepared_*/year=*/part-0.parquet")) #Locate all .parquet files by year. (This file path format is how the Tidy Finance blog post does it.)
all_data = {}  #Initialize an empty dictionary to store data keyed by year

for path in file_paths_for_neural_net_replication :  #Iterate over each file path of characteristics data by year
    chunk = pd.read_parquet(path)  #Read the parquet file into a pandas DataFrame
    year = int(path.split('=')[-1].split('\\')[0])  #Extract the year from the file path
    all_data[year] = chunk  #Store the year's data in the dictionary with the corresponding year as the key
    print(f"Loaded data for year: {year}")

characteristics_prepared_all = pd.concat(all_data.values(), ignore_index=True) #Concatenate all yearly DataFrames into one DataFrame
characteristics_prepared_all['year'] = pd.to_datetime(characteristics_prepared_all['date'], errors='coerce').dt.year #Create column in DataFrame for just year

gc.collect() #Manually free up memory

In [ ]:
#This cell trains the model for a given j OOS year.
#For the purposes of strictly replicating Gu et. al, full window is only range(0, 30, 1)

neural_net_replication_range = list(range(0, 30, 1)) #Range for replicating neural network from Gu. et al

for j in neural_net_replication_range: #Iterate through each OOS year. 30 years = to 2016
    print(f"\nProcessing iteration: {j+1}")

    split_dates = estimation_periods.iloc[j] #Get split dates for this OOS years
    cutoff_year = split_dates['validation_end'] #Get end year of validation window
    data = characteristics_prepared_all[characteristics_prepared_all['year'] <= cutoff_year] #Get data for training + validation
    train = data[data['year'] < split_dates['training_end']] #Define training set
    validation = data[(data['year'] >= split_dates['validation_start']) &  (data['year'] <= split_dates['validation_end'])] #Define validation set

    columns_to_drop = ['permno', 'date', 'mktcap_lag', 'year'] #Define list of columns to drop from predictors
    X_train = train.drop(columns=columns_to_drop + ['ret_excess']) #Prepare training inputs
    y_train = train['ret_excess'] #Prepare training outputs (ret_excess column contains risk premia)

    X_val = validation.drop(columns=columns_to_drop + ['ret_excess']) #Prepare validation inputs
    y_val = validation['ret_excess'] #Prepare validation outputs

    X_train = X_train.fillna(X_train.mean()) #Fill any N/A values in training inputs with the mean of its column
    X_val = X_val.fillna(X_val.mean()) #Fill any N/A values in training inputs with the mean of its column

    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32) #Convert training inputs into PyTorch tensor
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32) #Convert training output into PyTorch tensor
    X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32) #Convert validation inputs into PyTorch tensor
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32) #Convert validation output into PyTorch tensor

    train_ds = TensorDataset(X_train_tensor, y_train_tensor) #Create TensorDataaset for training data (inputs, output)
    val_ds = TensorDataset(X_val_tensor, y_val_tensor) #Create TensorDataaset for validation data (inputs, output)

    train_loader = DataLoader(train_ds, batch_size=10000, shuffle=True) #Create DataLoader for training data. Batch size is adjustable based on computational power/memory
    val_loader = DataLoader(val_ds, batch_size=10000) #Create DataLoader for validation data. Batch size is adjustable based on computational power/memory

    best_loss = float('inf') #Initialize best loss float
    best_model = None #Initialize best model container
    best_params = None #Initialize best model parameters container

    for params in grids: #Loop over all hyperparameter combinations in the grid
        model = NN3(input_dim=X_train.shape[1]) #Create model object using NN3 class
        trained_model, val_loss = train_model(model, train_loader, val_loader, lr=params['learning_rate'], weight_decay=params['penalty']) #Call "train_model()"
        if val_loss < best_loss: #If this train had a superior loss, save model as best model so far
            best_loss = val_loss
            best_model = trained_model
            best_params = params

    model_path = f"NN3_fitted_model_{split_dates['oos_year']}.pt" #Define file path for saving best model
    torch.save(best_model.state_dict(), model_path) #Save the best model for this OOS year to this file
    print(f"Saved model to {model_path}")


In [ ]:
#This cell makes OOS predictions using trained models for each OOS year
#Then, portfolio is constructed by going long on the top 10 percent of stocks by predicted risk premia and going short on the bottom 10 percent
#This is called the H-L portfolio. The table printed out at the bottom replicates Gu et. al methodology

neural_net_replication_oos_range = list(range(1987, 2017)) #1987-2016, OOS range for replicating Table 7 in Gu et. al paper
oos_predictions = [] #Initialize empty array for OOS predictions

for oos_year in range(1986, 2017):  #Iterate through each OOS year
    print(f"Processing OOS year: {oos_year}")

    feature_cols = [col for col in characteristics_prepared_all.columns if col not in ['permno', 'date', 'mktcap_lag', 'ret_excess', 'year']] #Get all predictor columns

    base_dir = r"YOUR_BASE_DIRECTORY_STORING_FITTED_MODELS"
    model_path = rf"{base_dir}\NN3_fitted_model_{oos_year}.pt" #Load the saved fitted model for this OOS year
    model = NN3(input_dim=len(feature_cols)) #Create model object using NN3 class
    model.load_state_dict(torch.load(model_path)) #Load model
    model.eval() #Specify model to be in eval mode, which is for validation or testing the model with out-of-sample data

    oos_data = characteristics_prepared_all[characteristics_prepared_all['year'] == oos_year].copy() #Get characteristics data from this OOS year
    oos_data.dropna(subset=['ret_excess'], inplace=True) #Drop any N/A values in the OOS data (shouldn't be any, but procedural assurance)

    X_oos = oos_data[feature_cols] #Get OOS input data in the feature columns
    X_oos = X_oos.fillna(X_oos.mean()) #Fill any N/A values in OOS inputs with the mean of its column
    X_oos_tensor = torch.tensor(X_oos.values, dtype=torch.float32) #Convert OOS input data to Tensor

    with torch.no_grad(): #Specify gradient tracking is not necessary
        preds = model(X_oos_tensor).squeeze() #Make model predictions

    oos_chunk = oos_data[['permno', 'date', 'mktcap_lag', 'ret_excess']].copy() #Get chunk of non-feature column data for this OOS year
    oos_chunk['predictions'] = preds.numpy() #Add the predictions column to this chunk for this OOS year
    oos_predictions.append(oos_chunk) #Add this year's chunk to the oos_predictions array

oos_predictions = pd.concat(oos_predictions, ignore_index=True) #Concatenate all year's data into one DataFrame

#Now, we will construct portfolios to replicate Gu et. al
def assign_portfolio_by_month(df, sorting_variable, n_portfolios=10): #This function will sort the DataFrame into 10 deciles based on a sorting variable, which in our case will be predicted risk premia
    df['portfolio'] = df.groupby('date')[sorting_variable].transform(lambda x: pd.qcut(x, q=n_portfolios, labels=False, duplicates='drop') + 1)
    return df

#Sort stocks into decile portfolios based on raw predicted returns (no value-weighting at this stage)
ml_portfolios = oos_predictions.copy() #Copy OOS predictions data
ml_portfolios = assign_portfolio_by_month(ml_portfolios, sorting_variable='predictions', n_portfolios=10) #Construct portfolio decile groups using assign_portfolio_by_month() function, sorting variable is predicted risk premia
ml_portfolios['portfolio'] = ml_portfolios['portfolio'].astype('category') #Convert numerical portfolio decile column into categorical variable
ml_portfolios.dropna(subset=['ret_excess'], inplace=True) #Removes rows where ret_excess has NaN values

def weighted_mean(df, value, weight): #This function calculates a weighted mean based off of a value and a corresponding weight.
    return (df[value] * df[weight]).sum() / df[weight].sum() #In our case, value = predicted risk premia (in percentage), weight = market capitalization
    #(to calculate realistic returns, we care more about high percentage gains/losses in bigger companies)

# Value-weighted monthly returns per decile
ml_portfolios_summary = ( #Calculate value-weighted monthly actual realized average returns and value-weighted predicted average returns and add columns to table
    ml_portfolios.groupby(['portfolio', 'date'], observed=False)
    .apply(lambda x: pd.Series({
        'predictions': weighted_mean(x, 'predictions', 'mktcap_lag'), #Add predicted average returns column (value-weighted)
        'ret_excess': weighted_mean(x, 'ret_excess', 'mktcap_lag') #Add realized average returns column (value-weighted)
    }))
    .reset_index()
)

#This code constructs the H-L portfolio as described in Gu et. al (long on top 10 percent predicted, short on bottom 10 percent predicted)

p10 = ml_portfolios_summary[ml_portfolios_summary['portfolio'] == 10].copy() #Copy the 10th decile (top decile in terms of raw percentage predicted risk premia)
p1 = ml_portfolios_summary[ml_portfolios_summary['portfolio'] == 1].copy() #Copy the 1st decile (bottom decile in terms of raw percentage predicted risk premia)

hl_merge = pd.merge(p10, p1, on='date', suffixes=('_10', '_1')) #Create DF by merging 10th decile and 1st decile
hl_merge['predictions'] = hl_merge['predictions_10'] - hl_merge['predictions_1'] #Long on 10th decile stocks, short on 1st decile stocks to measure theoretical value-weighted portfolio return (for predicted risk premia)
hl_merge['ret_excess'] = hl_merge['ret_excess_10'] - hl_merge['ret_excess_1'] #Long on 10th decile stocks, short on 1st decile stocks to measure theoretical value-weighted portfolio return (for realized risk premia)
hml_portfolio = hl_merge[['date', 'predictions', 'ret_excess']].copy() #Add date, value-weighted predictions, and value-weighted realized returns columns to H-L
hml_portfolio['portfolio'] = 'H-L' #Name: H-L

ml_portfolios_summary_updated = pd.concat([ml_portfolios_summary, hml_portfolio], ignore_index=True) #Combine 10 deciles with H-L portfolio in the table

overall_summary = ml_portfolios_summary_updated.groupby('portfolio').agg( #Construct overall summary table
    Pred=('predictions', 'mean'),
    Avg=('ret_excess', 'mean'),
    Std=('ret_excess', 'std')
).reset_index()

overall_summary['Pred'] *= 100 #Convert predicted risk premia to percentage format
overall_summary['Avg'] *= 100 #Convert realized risk premia to percentage format
overall_summary['Std'] *= 100 #Convert standard deviation of realized risk premia to percentage format
overall_summary['SR'] = ( #Calculate annualized Sharpe ratio
    overall_summary['Avg'] / overall_summary['Std']
) * (12 ** 0.5)

#Display table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(overall_summary)


In [ ]:
"""
THE FOLLOWING CELLS ARE THE CONTRIBUTION OF THIS PAPER
The following cells propose a portfolio optimization model, which aims to solve a customizable objective function using Sequential Least Squares Programming (SLSQP)
"""
from datetime import datetime #For dealing with dates
from scipy.optimize import minimize #For using minimize function, which is a general-purpose nonlinear solver
from sklearn.preprocessing import StandardScaler #For calculating Z-scored arbitrage difficulty penalty
from dateutil.relativedelta import relativedelta #For date arithmetic
import time #For dealing with time

OOS_OPT_START_YEAR = 1987 #Optimization year range begins here

CHARACTERISTIC_IMPORTANCE = { #Define importance weight for each stock-level DTA score interaction term (These importance weights were determined from analysis of existing literature)
    "ill": 1.00,
    "baspread": 1.00,
    "idiovol": 0.70,
    "turn": 0.50,
}

MACRO_IMPORTANCE = { #Define importance weight for each macroeconomic DTA score interaction term (These importance weights were determined from analysis of existing literature)
    "svar": 1.00,
    "tbl": 0.70
}

#Cartesian product of stock-level characteristics: (ill, idiovol, baspread, turn) and macroeconomic characteristics: (svar, tbl)
DTA_INTERACTION_MAP = { #Map characteristics with their two components
    "characteristic_ill_x_macro_svar": ("ill", "svar"),
    "characteristic_ill_x_macro_tbl": ("ill", "tbl"),
    "characteristic_idiovol_x_macro_svar": ("idiovol", "svar"),
    "characteristic_idiovol_x_macro_tbl": ("idiovol", "tbl"),
    "characteristic_baspread_x_macro_svar": ("baspread", "svar"),
    "characteristic_baspread_x_macro_tbl": ("baspread", "tbl"),
    "characteristic_turn_x_macro_svar": ("turn", "svar"),
    "characteristic_turn_x_macro_tbl": ("turn", "tbl"),
}

DTA_WEIGHTS = {} #Initialize empty dictionary for DTA_WEIGHTS

for interaction_term, (characteristic, macro) in DTA_INTERACTION_MAP.items(): #Iterate through each interaction term
    weight = CHARACTERISTIC_IMPORTANCE[characteristic] * MACRO_IMPORTANCE[macro] #Calculate weight of each interaction term
    DTA_WEIGHTS[interaction_term] = weight #Assign weight to interaction term

WINDOW_MONTHS = 12 #Define trailing window length for covariance matrix calculating (look 1 year back)
EPSILON = 1e-6 #Define smoothing constant for turnover penalty

print("Optimization Initialization completed.")

In [ ]:
def compute_di(df): #Function for computing the arbitrage difficulty penalty in the optimization problem at a given t
    Z = df[list(DTA_WEIGHTS.keys())].copy() #Extract the columns of the interaction terms involved in calculating the arbitrage difficulty penalty

    Z = Z.dropna() #Drop any rows with N/A values in these columns
    Z_scaled = pd.DataFrame( #Standardize each interaction term to ensure fair weighting across terms
        StandardScaler().fit_transform(Z), #Applies (x - mean) / std to each column
        index=Z.index,
        columns=Z.columns
    )

    weight_vector = np.array([DTA_WEIGHTS[col] for col in Z_scaled.columns]) #Get a weight vector aligned by columns/keys
    S = Z_scaled.values @ weight_vector #Calculate row-wise dot product
    D = (S - S.min()) / (S.max() - S.min()) #Normalize S to [0, 1]
    return pd.Series(D, index=Z_scaled.index) #Return final D_i vector

print("Arbitrage difficulty function defined.")

In [ ]:
def compute_cov_matrix(returns_df): #Function to calculate the covariance matrix of the stocks at a given t
    return np.cov(returns_df.T) #Return covariance matrix (transpose returns so that np.cov treats columns as each stock)

print("Covariance matrix function defined.")

In [ ]:
def optimization_objective(w, mu, Sigma, D, w_prev, lambda_t, lambda_r, lambda_a): #This function defines the optimization objective expression to be minimized
    #w = current weight vector (currently being optimized), mu = predicted risk premia output from model, Sigma = covariance matrix, D = computed arbitrage difficulty score [0, 1], w_prev: previous month's weights,
    #lambda_t: penalty weight for turnover, lambda_r: Penalty weight for risk, lambda_a: Penalty weight for arbitrage difficulty

    turnover = np.sum(np.sqrt((w - w_prev)**2 + EPSILON)) #Define smoothed turnover penalty that approximates |w_i - w_prev_i|.
    #Smoothed to insure differentiability (absolute value functions are indifferentiable at x=0, which is non-friendly to optimzation solvers)

    risk = w @ Sigma @ w #Define risk term as the transpose of the current weights multiplied by the covariance matrix, multiplied by the current weights
    #In this case, the weights vector in the leftmost operand are implicitly transposed.

    arb = np.sum(D * w**2) #Define weighted quadratic term discouraging heavy allocation to difficult-to-arbitrage stocks

    percentage_returns = w @ mu #Calculate the hypothetical percentage returns with the current set of weights
    return -(percentage_returns - lambda_t * turnover - lambda_r * risk - lambda_a * arb) #Return negative of the reward, since we use a minimizer

print("Optimization objective function defined.")

In [ ]:
def optimize_weights(mu, Sigma, D, w_prev, lambda_t, lambda_r, lambda_a, bounds=None): #Function to optimize the portfolio weights for one month (t)
    #mu = predicted risk premia output from model, Sigma = covariance matrix, D = computed arbitrage difficulty score [0, 1], w_prev: previous month's weights,
    #lambda_t: penalty weight for turnover, lambda_r: Penalty weight for risk, lambda_a: Penalty weight for arbitrage difficulty, bounds = Optional box constraints for each weight (could add some bounds to customize optimization)

    n = len(mu) #Get the current number of stocks we're considering
    w0 = np.zeros(n) #Initialize weights vector with all zeros
    constraints = ({ #Add constraint that Weights must sum to zero
        'type': 'eq',
        'fun': lambda w: np.sum(w)
    })

    def optimization_callback(w): #Function to be called at each iteration of SLSQP that displays the mean, max, and min of the current weight vector
        print(f"   ↳ Iteration: mean(w)={w.mean():.4e}, max={w.max():.4f}, min={w.min():.4f}")

    #This is where we start the optimization function
    print("Starting minimize()...")
    start_time = time.time() #Record start time of function

    res = minimize( #Call minimize() from scipy.optimize with the following arguments:
        optimization_objective, #Pass in the optimization objective function
        w0, #Pass in initial w0 of all zeros
        args=(mu, Sigma, D, w_prev, lambda_t, lambda_r, lambda_a), #Pass in tuple of additional arguments that are passed to the optimization objective, after w0.
        constraints=constraints, #Pass in constraints
        bounds=bounds, #Pass in bounds
        method='SLSQP', #Specify that we want to use SLSQP
        callback=optimization_callback, #Pass in callback function that will print out a small report after each iteration of optimization
        options={ #Pass in a set of options
            'disp': True, #We want final summary after convergence
            'maxiter': 100, #Must be large number such as 100 for meaningful results (could benefit from increasing, potentially.)
            'ftol': 1e-5 #Change of function value must be less than 10^-5 for us to stop function early
        }
    )

    elapsed = time.time() - start_time #Calculate elapsed time from minimize() call
    print(f"Optimization finished in {elapsed:.3f} seconds | Success: {res.success} | Iterations: {res.nit}\n") #Display elapsed seconds, success, and number of iterations
    if res.success : #If optimization problem succeeded
        return res.x #Return optimized weights
    else :
        print("------------FAILED: RETURNED ALL-ZERO WEIGHTS MATRIX DUE TO FAILURE TO OPTIMIZE------------") #Print failure message
        return np.zeros(n) #Return all-zero weights matrix

print("Optimization function defined.")

In [ ]:
def get_initial_HL_weights(mu_series): #Function to construct the initial w_prev vector using the H-L portfolio present in Gu et. al
    #mu_series = series object that contains permno as the ID's for predicted risk premia (series is similar to a dictionary)
    n = len(mu_series) #Get number of stocks currently working with
    n_in_decile = int(n * 0.1) #Get number of stocks in a decile

    mu_sorted = mu_series.sort_values(ascending=False) #Sort mu_series in descending order by its values, in this case predicted risk premia
    top_permnos = mu_sorted.iloc[:n_in_decile].index #Store top 10 percent of stocks in terms of predicted risk premia in top_permnos
    bottom_permnos = mu_sorted.iloc[-n_in_decile:].index #Store bottom 10 percent of stocks in terms of predicted risk premia in top_permnos

    long_weight = 1.0 / n_in_decile #Equal weight long position
    short_weight = -1.0 / n_in_decile #Equal weight short position

    weights = pd.Series(0.0, index=mu_series.index) #Initialize weights vector with all zeros
    weights.loc[top_permnos] = long_weight #Assign long_weight to top 10 percent
    weights.loc[bottom_permnos] = short_weight #Assign short_weight to bottom 10 percent

    assert np.isclose(weights.sum(), 0.0), "Weights do not sum to zero" #Ensure the sum of the initial weights is very close to 0

    return weights #Return the H-L portfolio weights

print("Initial H-L portfolio function defined.")

In [ ]:
#This cell prepares for the DataFrame for the portfolio optimization function
df_base = characteristics_prepared_all.copy() #Define DataFrame for the optimization problem
df_base['date'] = pd.to_datetime(df_base['date']) #Convert date column in DF to datetime objects
oos_predictions['date'] = pd.to_datetime(oos_predictions['date']) #Convert date column in oos_predictions to datetime objects

df_base = pd.merge( #Merge DF with oos_predictions
    df_base,
    oos_predictions[['permno', 'date', 'predictions']], #We care about these 3 columns in the oos_predictions DF: permno (stock ID), date, and risk premia prediction
    on=['permno', 'date'], #Merge on ID and date
    how='inner'
)

df_base = df_base[['permno', 'date', 'ret_excess'] + list(DTA_INTERACTION_MAP.keys()) + ['predictions']] #Filter DataFrame to only include permno, date, ret_excess, the arbitrage difficulty
df_base['month'] = df_base['date'].dt.to_period('M') #Add month column to DataFrame

print("DataFrame for portfolio optimization constructed.")

In [ ]:
def run_portfolio_optimization(lambda_t, lambda_r, lambda_a, full_oos=True, test_years=None): #Function for running portfolio optimization for a set of lambda penalty terms
    #lambda_t: penalty weight for turnover, lambda_r: Penalty weight for risk, lambda_a: Penalty weight for arbitrage difficulty
    #full_oos: boolean that tells function if we are running through whole OOS window (1987-2016)
    #test_years: if we are not running through the whole OOS window, this list specicies which years are to be tested

    monthly_dates = sorted(df_base['month'].unique()) #Get all unique monthly dates in whole DataFrame

    start_month = pd.Period(f'{OOS_OPT_START_YEAR}-01', freq='M') #Make start_month pd Period object based on the optimization start year

    if full_oos: #If caller wants to execute for the whole OOS period
        opt_months = [m for m in monthly_dates if m >= start_month] #Define opt_months as any month later or equal to start_month
    else: #If not
        opt_months = [m for m in monthly_dates if m.year in test_years] #Define opt_months as any month in the set of test_years

    print("Number of months to optimize: ", len(opt_months))
    print("First month: ", opt_months[0])
    print("Last month: ", opt_months[-1])

    weights_history = [] #Initialize weights_history to record the progression of weights for potential later analysis
    returns_history = [] #Initialize returns_history to record performance of optimized portfolio

    w_prev = pd.Series(dtype=float) #Initialize previous weights Series object

    for month in opt_months: #Iterate through each month in opt_months
        current_date = pd.Timestamp(month.to_timestamp()) #Get current date in months in timestamp format
        one_year_ago = pd.Timestamp((month - WINDOW_MONTHS).to_timestamp()) #Calculate the month one year ago in timestamp format

        past_returns = df_base[(df_base['month'] >= one_year_ago.to_period('M')) & (df_base['month'] < month)] #Filter DataFrame to include data between one year ago and now
        ret_matrix = past_returns.pivot_table(index='month', columns='permno', values='ret_excess', aggfunc='mean').dropna(axis=1) #Get matrix of past realized risk premia between one year ago and now

        if ret_matrix.shape[1] < 10 or ret_matrix.shape[0] < 12: #If there is not enough data for covariance matrix (SAFEGUARD, THIS SHOULD NEVER BE TRUE)
            print(f"------------Skipping {current_date.strftime('%Y-%m')} — insufficient data------------")
            continue

        aligned_permnos = [int(p) for p in ret_matrix.columns.tolist()] #Make list of aligned permnos (ID's for each stock) of only stocks that exist within return matrix of past 12 months
        Sigma = compute_cov_matrix(ret_matrix) #Call compute_cov_matrix by passing in all the actualized risk premia in the past 12 months for every stock
        Sigma = pd.DataFrame(Sigma, index=aligned_permnos, columns=aligned_permnos) #Convert the covariance matrix to a dataframe with the integer permnos on both axes

        current_df = df_base[df_base['month'] == month].copy() #Get this current months data from the DataFrame
        current_df['permno'] = current_df['permno'].astype(int) #Convert permnos to integer if they are not already
        current_df = current_df[current_df['permno'].isin(aligned_permnos)].dropna(  #Drop rows that have N/A values for any of the arbitrage difficulty interaction terms
            subset=list(DTA_INTERACTION_MAP.keys()) + ['ret_excess', 'predictions'] #or actualized risk premia or predicted risk premia (Shouldn't be any)
        )

        print(f"Processing {current_date.strftime('%Y-%m')}")

        if current_df.empty: #Should never happen, SAFEGUARD
            print(f"------------{current_date.strftime('%Y-%m')}: No valid stocks after filtering------------")
            continue

        current_df = current_df.set_index('permno').reindex(aligned_permnos).dropna().reset_index() #Align DataFrame with aligned_permnos and drop any N/A
        aligned_permnos = current_df['permno'].tolist() #Redefine aligned_permnos as all the permnos in DataFrame after alignment
        Sigma = Sigma.loc[aligned_permnos, aligned_permnos].values #Align covariance matrix with only values in aligned_permnos

        mu = current_df['predictions'].values #Get risk premia OOS predictions from the current month's data
        D = compute_di(current_df).reindex(current_df.index).fillna(0).values #Call compute_di to compute the arbitrage difficulty penalty for each permno

        if w_prev.empty: #If this is the first month iteration
            w_prev = get_initial_HL_weights(pd.Series(mu, index=aligned_permnos)) #Set previous weights to an initial H-L portfolio, similar to Gu et. al H-L portfolio
        else: #If this is not the first month iteration
            w_prev = w_prev.reindex(aligned_permnos).fillna(0.0) #Fill new stocks that were not present in the universe last month with 0.

        w_opt = optimize_weights(mu, Sigma, D, w_prev.values, lambda_t, lambda_r, lambda_a) #Call optimize_weights() to run optimization for this month

        #Now, we calculate performance metrics using the optimized portfolio weights
        realized_return = float(np.dot(w_opt, current_df['ret_excess'].values)) #Compute actual portfolio return as the dot product of optimized weights and the actualized stock returns
        predicted_return = float(np.dot(w_opt, mu)) #Compute predicted portfolio return as the dot product of optimized weights and predicted stock returns
        ex_ante_risk = float(np.dot(w_opt, np.dot(Sigma, w_opt))) #Compute ex-ante risk (expected portfolio variance, anticpated volatility of future returns) based on optimized weights and the covariance matrix.
        turnover = float(np.sum(np.abs(w_opt - w_prev.values))) #Compute portfolio turnover between optimized weights and last months optimized weights
        arb_exposure = float(np.dot(np.abs(w_opt), D)) #Compute arbitrage difficulty with optimized weights (weights are not squared like they are in optimization objective, that is by design, we don't want this performance metric to be quadratic)

        weights_history.append(w_opt) #Add the optimized weights to the weights_history array
        returns_history.append({ #Add this month's performance metrics dictionary to the returns_history array
            'date': current_date,
            'return': realized_return,
            'predicted': predicted_return,
            'ex_ante_risk': ex_ante_risk,
            'turnover': turnover,
            'arb_exposure': arb_exposure
        })

        w_prev = pd.Series(w_opt, index=aligned_permnos) #Re-define previous weights as the current optimized weights for next iteration

        print(f"Optimized for {current_date.strftime('%Y-%m')}")

    returns_df = pd.DataFrame(returns_history)
    returns_df = returns_df.sort_values(by='date').reset_index(drop=True) #Sort metrics DataFrame by date
    return returns_df #Return metrics DataFrame

print("Optimization runner function defined")


In [ ]:
def compute_optimized_summary(returns_df): #Function to build the summary table for optimized portfolio
    df = returns_df.copy() #Copy DataFrame into new container

    df['return'] = df['return'].astype(float) #Convert returns column to float
    df['predicted'] = pd.to_numeric(df.get('predicted'), errors='coerce') #Convert predictions column to numeric
    df['ex_ante_risk'] = pd.to_numeric(df.get('ex_ante_risk'), errors='coerce') #Convert ex-ante risk column to numeric
    df['turnover'] = pd.to_numeric(df.get('turnover'), errors='coerce') #Convert turnover column to numeric
    df['arb_exposure'] = pd.to_numeric(df.get('arb_exposure'), errors='coerce') #Convert arb_exposure column to numeric

    predicted_mean = df['predicted'].mean(skipna=True) * 100 #Get the percentage mean of the predicted column
    realized_mean = df['return'].mean() * 100 #Get the percentage mean of the realized column
    realized_sd = df['return'].std() * 100 #Compute the standard deviation column of realized returns
    sharpe_ratio = (realized_mean / realized_sd) * np.sqrt(12) #Compute the annualized Sharpe Ratio (risk-adjusted performance metric)
    avg_ex_ante_risk = df['ex_ante_risk'].mean(skipna=True) * 100 #Get the percentage mean of the ex-ante risk column
    avg_turnover = df['turnover'].mean(skipna=True) * 100 #Get the percentage mean of the turnover column
    avg_arb_exposure = df['arb_exposure'].mean(skipna=True) #Get the percentage mean of the arbitrage difficulty exposure column

    summary_df = pd.DataFrame({ #Build the final summary table with all these performance metrics
        'portfolio': ['Optimized'],
        'predicted_mean': [predicted_mean],
        'realized_mean': [realized_mean],
        'realized_sd': [realized_sd],
        'sharpe_ratio': [sharpe_ratio],
        'avg_ex_ante_risk': [avg_ex_ante_risk],
        'avg_turnover': [avg_turnover],
        'avg_arb_exposure': [avg_arb_exposure]
    })

    return summary_df #Return the summary table

def get_filename(lt, lr, la): #Function to build filename for storing the results for a lambda combination
    return f"lambda_t{lt}_r{lr}_a{la}.pkl" #Save in simple .pkl file

print("Summary function defined.")


In [ ]:
#This cell is where we test performance for different combinations of lambda penalty values for turnover, risk, and arbitrage difficulty
lambda_configs = [ #Array to store all the combinations of lambdas we want to try
    (0.6, 0.5, 0.6), #Neutral penalties (pension fund-ish)
    (0.3, 0.2, 0.3), #Liberal penalties (hedge fund-ish)
    (1.0, 1.2, 1.0), #Conservative penalties (retail investor-ish)
    (1.0, 0.7, 1.5), #Conservative penalties (conservative mutual fund-ish)
]

#Now we will add the cartesian product of the following possible lambdas
lambda_t_values = [0.3, 0.6, 1.0] #Turnover lambdas we want to try
lambda_r_values = [0.2, 0.5, 1.0] #Risk lambdas we want to try
lambda_a_values = [0.3, 0.6, 1.0] #Arb difficulty lambdas we want to try

lambda_configs = lambda_configs + list(product(lambda_t_values, lambda_r_values, lambda_a_values)) #Add the cartestian product of the possible lambdas 2 x 3 x 3 = 18 additional configs, 22 total

for i, (lt, lr, la) in enumerate(lambda_configs): #Iterate through each lambda combination'
    filename = get_filename(lt, lr, la) #Make filename string using these lambdas

    print(f"Running λ_t={lt}, λ_r={lr}, λ_a={la}")

    #Check if file already exists
    if os.path.exists(filename):
        print(filename, "already exists. Skipping.")
        continue

    returns_df = run_portfolio_optimization(lt, lr, la, full_oos=False, test_years=[2004, 2005, 2008, 2009]) #Run portfolio optimization for this lambda combination
    #We pick 2004, 2005, 2008, 2009 to test on a mix of smooth and rocky market conditions

    summary_df = compute_optimized_summary(returns_df) #Make summary table for this lambda combination

    result = { #Construct result dictionary containing which lambdas we're on and their corresponding summary table
        'lambdas': {'lambda_t': lt, 'lambda_r': lr, 'lambda_a': la},
        'summary': summary_df
    }

    with open(filename, "wb") as f: #Store result dictionary into a .pkl file
        pickle.dump(result, f)

    print(filename, " was saved successfully.")

In [ ]:
folder_path = r"YOUR_FOLDER_PATH_STORING_PKL_FILES"
file_names = os.listdir(folder_path) #Put all .pkl files in a folder and list of all them here

results = {} #results dictionary to store results of all lambda combinations
for file_name in file_names: #Iterate through each .pkl filename
    full_path = os.path.join(folder_path, file_name) #Get full path
    with open(full_path, "rb") as f: #Open file
        results[file_name] = pickle.load(f) #Add to results dictionary

all_summaries = [] #Build metrics DataFrame
for file_name, result in results.items(): #Iterate through each lambda's value in results
    s = result['summary'].iloc[0].copy() #Get summary section
    s['file_name'] = file_name  #Store file name as plain string
    all_summaries.append(s)

df_metrics = pd.DataFrame(all_summaries) #Make actual DF

#For normalizing metrics
metric_info = {
    'realized_mean': {'invert': False}, #Better if higher
    'sharpe_ratio': {'invert': False}, #Better if higher
    'avg_ex_ante_risk': {'invert': True}, #Better if lower
    'avg_turnover': {'invert': True}, #Better if lower
    'avg_arb_exposure': {'invert': True}, #Better if lower
}

for metric, info in metric_info.items(): #Iterate through each metric
    col = df_metrics[metric] #Get the column for the metric
    normalized = (col - col.min()) / (col.max() - col.min()) #Normalize
    if info['invert']:
        normalized = 1 - normalized #Invert if necessary
    df_metrics[f'norm_{metric}'] = normalized #Store

#Dictionary mapping investor types to their relative priority (1–10) for each portfolio performance metric.
#These weights reflect how much each investor group values metrics like realized return, Sharpe ratio,
#ex-ante risk, turnover, and arbitrage exposure. Higher numbers indicate stronger preferences.
investor_priorities = {
    'retail_investor': {
        'realized_mean': 3,
        'sharpe_ratio': 5,
        'avg_ex_ante_risk': 9,
        'avg_turnover': 8,
        'avg_arb_exposure': 6
    },
    'pension_fund': {
        'realized_mean': 6,
        'sharpe_ratio': 8,
        'avg_ex_ante_risk': 6,
        'avg_turnover': 5,
        'avg_arb_exposure': 3
    },
    'mutual_fund': {
        'realized_mean': 5,
        'sharpe_ratio': 6,
        'avg_ex_ante_risk': 5,
        'avg_turnover': 5,
        'avg_arb_exposure': 8
    },
    'hedge_fund': {
        'realized_mean': 10,
        'sharpe_ratio': 10,
        'avg_ex_ante_risk': 2,
        'avg_turnover': 2,
        'avg_arb_exposure': 2
    }
}

def score_config(row, group): #Define scoring function
    weights = investor_priorities[group] #Get weights for investor
    score = 0 #Set score to 0
    for metric in weights: #Iterate through each of this group's metric and weight
        score += weights[metric] * row[f'norm_{metric}'] #Add weighted score for this metric to overall score
    return score #Return final score

df_metrics.reset_index(drop=True, inplace=True)

for group in investor_priorities.keys(): #Iterate through each group
    df_metrics[f'{group}_score'] = df_metrics.apply(lambda row: score_config(row, group), axis=1) #Get final score for each lambda combination for this group

    best_idx = df_metrics[f'{group}_score'].idxmax() #Get best score
    best_file = df_metrics.loc[best_idx, 'file_name']  #Get filename for best score

    print(f"\nBest config for {group}:") #Print results
    print(f"Lambdas: {results[best_file]['lambdas']}")
    print(f"Summary:\n{results[best_file]['summary']}")

#Extract lambdas from filename for clarity in the table
df_metrics['lambda_t'] = df_metrics['file_name'].apply(lambda x: float(x.split('_r')[0].replace('lambda_t', '').replace('.pkl', '').replace('a', '')))
df_metrics['lambda_r'] = df_metrics['file_name'].apply(lambda x: float(x.split('_r')[1].split('_a')[0]))
df_metrics['lambda_a'] = df_metrics['file_name'].apply(lambda x: float(x.split('_a')[1].replace('.pkl', '').replace('\n', '')))

columns_to_display = [ #Keep only relevant columns for the final output
    'lambda_t', 'lambda_r', 'lambda_a',
    'realized_mean', 'sharpe_ratio', 'avg_ex_ante_risk',
    'avg_turnover', 'avg_arb_exposure'
]

df_metrics = df_metrics.sort_values(by=['lambda_t', 'lambda_r', 'lambda_a']).reset_index(drop=True) #Sort

print("\n=== Results Table: All Lambda Configurations ===\n")
print(df_metrics[columns_to_display].to_string(index=False)) #Print full table

#Thank you! Please reach out with any questions at reginaldjhyde@gmail.com or hyde9698@stthomas.edu